## Live Demo
---

1. Load 3D image.

In [ ]:
import ctwidgets

mouse = ctwidgets.load_image("CT280.gff")
print(mouse.data.shape) # dimensions

2. Extract 3D image (one timepoint).

In [ ]:
mouse_3d = mouse.get_timepoint(0)

3. Launch interactive visualization widget.

In [ ]:
app = ctwidgets.SliceWidget(mouse_3d)

4. Load manual segmentation.

In [ ]:
seg = ctwidgets.load_image("Organs.segff")
print(seg.data.info)
print(seg.metadata)

5. Visualize again, now in color.

In [ ]:
app2 = ctwidgets.SliceWidget(seg.get_timepoint(0), colored=True)

# add class names to colorbar ticks
class_dict = seg.get_class_names()
names = list(class_dict.keys())
indices = list(class_dict.values())
app2.add_class_names(names, indices)

- Pretrained deep learning model for 3D lung tumor segmentation: [nnU-Net](https://github.com/MIC-DKFZ/nnUNet)
    - Published and well-documented.
    - Winner of the 2018 [*Medical Segmentation Decathlon*](http://medicaldecathlon.com/) challenge (public datasets with exper labels).
    
    - Difficulties:
        - Training a model on new data myself is not feasible.
        - Inference is computationally intense.
        - Linux only (!)
        - No validation dataset for challenge (!).

    - Strategy for `ctwidgets`:
        - Segmentations are precomputed.
        - New widget **SegmentationWidget** will offer visualization of dataset + segmentation (+ validation segmentation).
        - If validation segmentation is present, Dice score can be computed.